<h2> Liora Wachsstock </h2>
<h3> Project 2 Python for Data Science </h3>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# constants
UPPER = 10
LOWER = -5
NUM_ITERATIONS = 1000
d = 5
n = 20

In [3]:
def generate_data(seed_num, d, n):
    
    rng = np.random.default_rng(seed = seed_num)

    data = (UPPER - LOWER) * rng.random((n,d)) + LOWER # "uniformly randomly, obeying the bounds"
    
    return pd.DataFrame(data)

def random_parameters(seed_num, num):
    rng = np.random.default_rng(seed = seed_num)
    return rng.random((num, 2)) #uniform random, two for each d

def f(x):
    return np.square(np.sum(np.square(x)))

def best(data):
    return data[np.argmin(f(data))]

def worst(data):
    return data[np.argmax(f(data))]
    
def new_vector(current_i, best_i, worst_i, r1, r2):
    new_i = current_i + r1*(best_i - abs(current_i)) - r2*(worst_i - abs(current_i))
    
    if (new_i < LOWER):
        return LOWER
    elif (new_i > UPPER):
        return UPPER
    else:
        return new_i

    
def algo(df, num_iterations):
    data = df.T # its way more intutive to iterate through the rows, so transpose the dataframe
    
    updates = 0
    while (updates < num_iterations):
        r = random_parameters(42, data.shape[0])
        best_v = best(data)
        worst_v = worst(data)
        
        for i, vector in enumerate(data): #updating every vector in data (this is the only "for" loop in program)
            vfunc = np.vectorize(new_vector)
            new_v = vfunc(vector, best_v, worst_v, r.T[0], r.T[1]) 
            
            data[i] = new_v if (f(new_v) <= f(data[i])) else data[i] #if the new vector is at least as good, update it
            
        updates += 1
        
        #if any vector hits the global minimum, stop
        if (not data.any().all()):
            break
    
    return data.T, updates



In [4]:
df = generate_data(255, d, n)
df, iters = algo(df, NUM_ITERATIONS) #run the alogrithm 

In [5]:
print("1. Population size:", n)
print("2. Numerical values of d that cause the minium value of the function:\n", np.array(best(df.T)))
print("3. Corresponding Minimum value of the function: ", f((best(df.T))))
print("4. Max iterations used: ", iters)


df = df.rename(index =lambda x: 'x'+str(x), 
               columns=lambda x: 'd'+str(x))

print("5.\n First five frames:")
print(df.head()) #first 5 frames
print("   \nLast Five frames:")
print(df.tail())#last 5 frames 


1. Population size: 20
2. Numerical values of d that cause the minium value of the function:
 [-1.82870076 -1.71924942  0.97016645 -0.13884091  1.11725567]
3. Corresponding Minimum value of the function:  72.398400995388
4. Max iterations used:  1000
5.
 First five frames:
          d0        d1        d2        d3        d4
x0 -5.000000  0.000000  0.000000 -5.000000 -3.000000
x1 -5.000000  0.000000  1.000000 -4.000000 -1.000000
x2 -4.374295  0.690940  2.968148 -3.613887 -0.598365
x3  2.150052  4.888987  0.139176 -2.580104 -0.008286
x4 -3.044450  2.368480  6.731038 -1.564038  2.046180
   
Last Five frames:
           d0        d1        d2        d3        d4
x15  3.061387  4.393667  2.270637  6.087378  2.283492
x16  5.848212 -2.285198 -2.015647 -2.424137  8.444106
x17  5.909014  5.672367 -1.593297 -1.281464 -1.650901
x18 -3.279895  0.641005  7.704547 -1.865118 -0.578825
x19  3.686700 -4.752026 -0.140338  4.024761  3.702227
